In [1]:
# Train number of different models from Flair framework.
# With different sized trainin data
# save predictions of each model to file

# Notice - 1st run may take long as model weights are downloaded

In [2]:
# Dataset
# https://github.com/t-davidson/hate-speech-and-offensive-language

# Paper
# https://aaai.org/ocs/index.php/ICWSM/ICWSM17/paper/view/15665

# Their code
# https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/src/Automated%20Hate%20Speech%20Detection%20and%20the%20Problem%20of%20Offensive%20Language%20Python%203.6.ipynb

In [3]:
# Code based on https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_4_ELMO_BERT_FLAIR_EMBEDDING.md

In [5]:
import pandas as pd
import random
import os
import numpy as np
import torch

import nltk
import string
import re
from nltk.stem.porter import *
import time

# Display whole text of dataframe field and don't cut it
pd.set_option('display.max_colwidth', -1)

In [6]:
print(f'torch version: {torch.__version__}')

torch version: 1.3.1


In [7]:
dataset = 'hatespeech'

current = os.getcwd()
basefolder = current + '/dataset_'+ dataset+'/'
datafolder = basefolder + 'data/'  # for example /dataset_businessnews/data/
print(basefolder)

infolder =  basefolder + 'input/'
outfolder = basefolder + 'output/'

/home/max/git/modelcompare/dataset_hatespeech/


In [8]:
from flair.data import Sentence
from flair.data_fetcher import NLPTaskDataFetcher

from flair.embeddings import WordEmbeddings, StackedEmbeddings, DocumentRNNEmbeddings
from flair.embeddings import DocumentPoolEmbeddings
from flair.embeddings import FlairEmbeddings, BertEmbeddings, ELMoEmbeddings
from flair.embeddings import BytePairEmbeddings

from flair.embeddings import OpenAIGPTEmbeddings
#from flair.embeddings import OpenAIGPT2Embeddings

# New DocumentRNNEmbeddings, deprecates DocumentLSTMembeddings
# from flair.embeddings import #DocumentLSTMEmbeddings

from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

# new ones: GPT-1 and GPT-2
# https://github.com/flairNLP/flair/tree/master/resources/docs/embeddings
# https://github.com/flairNLP/flair/blob/master/resources/docs/embeddings/TRANSFORMER_EMBEDDINGS.md

/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-pack

In [9]:
SEED = 1
# REPEATABILITY
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything() # also called here

# TEXT PREPROCESS
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x: # comparison makes faster
            x = x.replace(punct, f' {punct} ')
    return x

quotes = ['″', '′', '"'] # apostrophe "'"
def mark_quotes(x):
    x = str(x)
    for quote in quotes:
        if quote in x: # comparison makes faster
            x = x.replace(quote, f'quote')
    return x

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    
    mention_regex = '@[\w\-]+'
    
    #add #, mention, e.g. &#8120     
    mention_regex2  =   '&#[0-9]*' 
    
    
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, ' URL ', parsed_text)
    parsed_text = re.sub(mention_regex, ' MENTION', parsed_text)
    parsed_text = re.sub(mention_regex2, ' MENTION', parsed_text)    
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    # *needed to be removed or outputs a list of letters
    #tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    tweet = " ".join(re.split("[^a-zA-Z.,!?]", tweet.lower())).strip()  
    #tweet = " ".join(re.split(r'\s+', tweet.lower())).strip()
    return tweet.split()

In [10]:
def loadData():
    train = pd.read_csv(basefolder+'input/train.csv',sep='\t', header = None)
    dev = pd.read_csv(basefolder+'input/dev.csv'  ,sep='\t', header = None)
    test = pd.read_csv(basefolder+'input/test.csv'  ,sep='\t', header = None)    
    train.columns = ['id','label','text']
    dev.columns  = ['id','label', 'text']
    test.columns  = ['id','label', 'text']   
    return train, dev, test

In [11]:
def preprosess_hatespeech(df):
    df.text = df.text.apply(lambda x: preprocess(x)) #URL, @mention etc
    df["text"] = df["text"].apply(lambda x: clean_text(x))
    df["text"] = df["text"].apply(lambda x: mark_quotes(x))
    #df.text = df.text.apply(lambda x: basic_tokenize(x))
    return df

In [12]:
# Turn label from digit into Fasttext format __label__.  "1" into "__label__1"
def toFasttext(df):
    df['label'] = '__label__' + df['label'].astype(str)
    return df

In [13]:
data_folder = infolder

In [14]:
# randomize

# Load data
train, dev, test = loadData()

train = train.iloc[np.random.permutation(len(train))]


#Test smaller
trainsize = len(train)

'''SET TRAINSIZE HERE'''
# 100, 200, 500, 1k, 3k, 7k, 18k
trainsize = 100
train = train[0:trainsize]

print(len(train))
print(len(dev))
print(len(test))

100
3000
3000


### Preprocess

In [15]:
train = preprosess_hatespeech(train)
dev = preprosess_hatespeech(dev)
test = preprosess_hatespeech(test)

In [16]:
train = toFasttext(train)
dev = toFasttext(dev)
test = toFasttext(test)

In [17]:
train.head()

,id,label,text
15131,18202,__label__1,RT MENTION : MENTION That has redneck written all over it lol . Drunks can use gravity to get down MENTION ;
4106,3986,__label__2,"MENTION her face ugly to me , & amp ; her nudes were trash ."
3365,4483,__label__1,MENTION come draw and paint with me niggah !
6004,12574,__label__1,Krakin this bitch ipem at 12 . . . WHO SIPPIN WIT ME . . . ELEMENTS URL
11261,10768,__label__1,I hate when bitches quote been thinking quote smh


In [18]:
# Id in start of line is not Fasttext format: remove id
train.drop(['id'], axis=1, inplace=True)
dev.drop(['id'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)

# Write to Flairs input csv files
train.to_csv(basefolder+'input/flair_train.csv',sep='\t', index = False, header = False)
dev.to_csv(basefolder+'input/flair_dev.csv'  ,sep='\t', index = False, header = False)
test.to_csv(basefolder+'input/flair_test.csv',sep='\t', index = False, header = False)

### Flair

### Stacked embeddings


In [19]:
# Embeddings
# https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_4_ELMO_BERT_FLAIR_EMBEDDING.md

# 'multi-forward', multi-lang English, German, French, Italian, Dutch, Polish, 
#        Mix of corpora (Web, Wikipedia, Subtitles, News)

# 'mix-forward'English,   Forward LM embeddings over mixed corpus (Web, Wikipedia, Subtitles)

`StackedEmbeddings` are currently a `WordEmbeddings` class, so they cannot directly be used to classify 
documents. They can only be used for sequence labeling.

However, you can put a stack of word embeddings into one of the `DocumentEmbeddings` classes such as `DocumentPoolEmbeddings` or `DocumentLSTMEmbeddings`. This way, you are specifying how to aggregate word embeddings for text classification

So `DocumentPoolEmbeddings` will simply average them, while `DocumentLSTMEmbeddings` will train an LSTM over them.

 https://github.com/zalandoresearch/flair/issues/414
 
 *update depracated: DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings')

In [20]:
# OR average 
# document_embeddings = DocumentPoolEmbeddings(word_embeddings)

#### Model

In [21]:
# https://towardsdatascience.com/text-classification-with-state-of-the-art-nlp-library-flair-b541d7add21f

# https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md

In [22]:
#data_folder = infolder
data_folder

'/home/max/git/modelcompare/dataset_hatespeech/input/'

In [23]:
corpus = NLPTaskDataFetcher.load_classification_corpus(data_folder, test_file='flair_test.csv', dev_file='flair_dev.csv', train_file='flair_train.csv')

2020-06-23 11:58:35,988 Reading data from /home/max/git/modelcompare/dataset_hatespeech/input
2020-06-23 11:58:35,989 Train: /home/max/git/modelcompare/dataset_hatespeech/input/flair_train.csv
2020-06-23 11:58:35,991 Dev: /home/max/git/modelcompare/dataset_hatespeech/input/flair_dev.csv
2020-06-23 11:58:35,992 Test: /home/max/git/modelcompare/dataset_hatespeech/input/flair_test.csv


/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  """Entry point for launching an IPython kernel.
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:452: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, tokenizer=tokenizer, max_tokens_per_doc=max_tokens_per_doc
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:457: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, tokenizer=tokenizer, max_tokens_per_doc=max_tokens_per_doc
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packag

In [24]:
print(corpus)
#print(len(corpus.train))

# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()
#print(label_dict)

Corpus: 100 train + 3000 dev + 3000 test sentences
2020-06-23 11:58:51,988 Computing label dictionary. Progress:


100%|██████████| 100/100 [00:00<00:00, 79724.46it/s]

2020-06-23 11:58:52,005 [b'1', b'2', b'0']


In [25]:
### Individual model only, skip when in batch

In [26]:
# classifier = TextClassifier(stacked_embeddings, label_dictionary=corpus.make_label_dictionary(),
#                            multi_label=False)

In [27]:
# Glove alone 0.902

In [28]:
# Winners: Fasttext: en-crawl, 0.9129 solo

# word embeddings pooled:glove+fasttext 'test_score': 0.8892,

# glove, fasttext:en-crawl, flair-multi-forward, elmo('original') f1-score 0.9155
# ELMO(original) test_score': 0.9002 (f-score 0.9145 )
# flair-news-forward WITHOUT TOKENIZATION, 'test_score': 0.8526
# flair-news-forward: 'test_score': 0.8741
# BERT only + RNN 256, bidir=True, 'test_score': 0.904,
# Tweet + Bert same, not very great, used 128 rnn?
#.
# 0.8911 flair + BPE


##############################

#NEW BEST adding preprocess puncts+quotes, glove, en-twitter, en-crawl:   test_score': 0.9242,
# no stemming


# BEST glove, en-twitter, en-crawl, flair-multi-forward:  f1-score 0.9129
# add stemming f-score 0.7808   Breaking down, very bad score



# ---------------------

# Added Preprosessing, no stemming
# 'test_score': 0.9072 glove, en-twitter , flair-multi-forward

# Add to this, en-crawl Fastext  # f1-score 0.9129 BEST

# Adding Kominov embedding: 'test_score': 0.8868 -> weaker considerably, by 0.03


# ----------------------------
# Compare single embeddings
# en-twitter          'test_score': 0.8943
# en-crawl, Fasttext 'test_score': 0.9064,
# flai-multi-forward: 'test_score': 0.7738  WEAK! WHY? Also slow


# ----------------------------

# Glove, en-twitter, en-crawl: 'test_score': 0.8865,  (20 epoch)
# Glove, en-twitter, en-crawl: same + 2-way FLAIR  'test_score': 0.8854 - weaker, but 15 epoch

# bert only, no glove: slow, 'test_score': 0.7329
# Flair only, no glove: 'test_score': 0.8158  (0.06 lower than with Glove)

# Glove, Flair-multi 2ways, Bert-base-cased : OOM

# test_score': 0.8763, Glove + doc-embedding multi 

# test_score': 0.8731, Glove + doc-embedding news


In [29]:
# trainer.train('./', max_epochs=10)

### Predict

In [30]:
def saveResults(savelist, name='all_default'):
    import shelve
    # file to be used
    filename= name+'.shlf'
    shelf = shelve.open(outfolder+filename)
    #shelf = shelve.open("all_flair.shlf")
    # serializing
    #shelf["all_flair"] = all_flair
    shelf[name] = savelist
    shelf.close() # you must close the shelve file!!!

In [31]:
def loadResults(name='all_default'):
    import shelve
    filename= name+'.shlf'
    shelf = shelve.open(outfolder+filename) 
    new = shelf[name]
    shelf.close()
    return new

In [32]:
''' Train with each embedding in the list, predict, add results to the list

Parameters: word_embeddings,    eg   WordEmbeddings('glove')
            modelname and modeldesc - text to be added in results
            savelist : list where results are appended. Can be empty or already including results
            epohcs: epochs to run

'''


def train_and_predict(embeddings, modelname, modeldesc, savelist, epochs=15, batch_size=32):
    
    print(modelname)
    start = time.time()
    
# PREPARE
    document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512,
                                           bidirectional = False,
                                           rnn_type='LSTM', 
                                           reproject_words=True, reproject_words_dimension=256                                                    
                                           )

    seed_everything(SEED)
    corpus = NLPTaskDataFetcher.load_classification_corpus(data_folder, test_file='flair_test.csv', dev_file='flair_dev.csv', train_file='flair_train.csv')

    seed_everything(SEED)
    label_dict = corpus.make_label_dictionary()

    seed_everything(SEED)
    classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(),
                            multi_label=False)
    trainer = ModelTrainer(classifier, corpus)
    
# TRAINING
    seed_everything(SEED)
    trainer.train('./', 
              learning_rate=0.1,
              mini_batch_size=batch_size, # 32  # BERT OOM even with 16 batch -> need 8. others run on 32 or even more
              anneal_factor=0.5,
              patience=5,     
              max_epochs=epochs,  #15
              ) #max_epochs=150

    duration_train = time.time()-start
    
# PREDICT
    start_pred = time.time()

    # turn text into Flairs "Sentence object"
    test['flair_sentence'] = test['text'].apply(lambda x: Sentence(x))

    # discard output, result is put into object itself
    _ = test['flair_sentence'].apply(lambda x: classifier.predict(x))

    # sentence.labels returns a list containing flairs Label object that includes a dict. 
    # dig the values for predicted label + confidence from within the dict
    # the 'value' returns a str, cast it to int
    test['yhat'] = test['flair_sentence'].apply(lambda x: int(x.labels[0].to_dict()['value']))

    test['confidence'] = test['flair_sentence'].apply(lambda x: x.labels[0].to_dict()['confidence'])

    results = pd.DataFrame(test[['yhat', 'confidence']])
    results.columns=['label','confidence']
    results.head()
    
# ADD RESULTS TO LIST

    duration_predict = time.time() - start_pred
    #print(f'Duration {duration:.2f} s')

    savelist.append({'model': modelname,
                'labels': results['label'],
                'confidence': results['confidence'],
                'traintime': duration_train,
                'predtime3k': duration_predict,
                'modeldesc': modeldesc
               }
              )

### function mode


In [33]:
savelist = [] # list to save results
modeldesc = '512LSTM_15epoch_non-bi'
#BATCH_SIZE=32
EPOCHS = 15

#### To use ELMoEmbeddings, please first install with "pip install allennlp"

In [34]:
total_time = time.time()

word_embeddings = [ WordEmbeddings('glove'),              ]
modelname = 'glove'
train_and_predict(word_embeddings, modelname, modeldesc, savelist=savelist, epochs=EPOCHS)

word_embeddings = [ WordEmbeddings('en-crawl'),                 ]
modelname = 'fasttext web-crawl'
train_and_predict(word_embeddings, modelname, modeldesc, savelist=savelist, epochs=EPOCHS)

word_embeddings = [ WordEmbeddings('en'),                 ]
modelname = 'fasttext news/wiki'
train_and_predict(word_embeddings, modelname, modeldesc, savelist=savelist, epochs=EPOCHS)

word_embeddings = [ WordEmbeddings('en-twitter'),                 ]
modelname = 'en-twitter'
train_and_predict(word_embeddings, modelname, modeldesc, savelist=savelist, epochs=EPOCHS)

word_embeddings = [ ELMoEmbeddings('original')              ]
modelname = 'elmo'
train_and_predict(word_embeddings, modelname, modeldesc, savelist=savelist, epochs=EPOCHS)

print(time.time() - total_time)

# 1289 sec on 100 train (was 2x elmo)


# 2403 sec on 18k
# 2684 s   18 k

glove
2020-06-23 11:59:33,094 Reading data from /home/max/git/modelcompare/dataset_hatespeech/input
2020-06-23 11:59:33,095 Train: /home/max/git/modelcompare/dataset_hatespeech/input/flair_train.csv
2020-06-23 11:59:33,096 Dev: /home/max/git/modelcompare/dataset_hatespeech/input/flair_dev.csv
2020-06-23 11:59:33,097 Test: /home/max/git/modelcompare/dataset_hatespeech/input/flair_test.csv


/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:452: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, tokenizer=tokenizer, max_tokens_per_doc=max_tokens_per_doc
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:457: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, tokenizer=tokenizer, max_tokens_per_doc=max_tokens_per_doc
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:464: DeprecationWarning:

2020-06-23 11:59:34,817 Computing label dictionary. Progress:


100%|██████████| 100/100 [00:00<00:00, 168988.88it/s]

2020-06-23 11:59:34,821 [b'1', b'2', b'0']
2020-06-23 11:59:34,821 Computing label dictionary. Progress:


...

2020-06-23 12:11:11,675 [b'1', b'2', b'0']
2020-06-23 12:11:11,678 ----------------------------------------------------------------------------------------------------
2020-06-23 12:11:11,679 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): ELMoEmbeddings(model=0-elmo-original)
    )
    (word_reprojection_map): Linear(in_features=3072, out_features=256, bias=True)
    (rnn): LSTM(256, 512, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=3, bias=True)
  (loss_function): CrossEntropyLoss()
)"
2020-06-23 12:11:11,680 ----------------------------------------------------------------------------------------------------
2020-06-23 12:11:11,682 Corpus: "Corpus: 100 train + 3000 dev + 3000 test sentences"
2020-06-23 12:11:11,682 ----------------------------------------------------------------------------------------------------
2020-06-23 12

2020-06-23 12:11:11,945 epoch 1 - iter 0/4 - loss 1.20414293 - samples/sec: 129.35
2020-06-23 12:11:12,243 epoch 1 - iter 1/4 - loss 0.90223247 - samples/sec: 112.48
2020-06-23 12:11:12,467 epoch 1 - iter 2/4 - loss 0.88235418 - samples/sec: 146.83
2020-06-23 12:11:12,535 epoch 1 - iter 3/4 - loss 0.85789797 - samples/sec: 520.09
2020-06-23 12:11:12,546 ----------------------------------------------------------------------------------------------------
2020-06-23 12:11:12,547 EPOCH 1 done: loss 0.8579 - lr 0.1000
2020-06-23 12:11:31,969 DEV : loss 0.6743201017379761 - score 0.7667
2020-06-23 12:11:32,118 BAD EPOCHS (no improvement): 0


/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ELMoEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type _ElmoBiLm. It won't be c

2020-06-23 12:11:33,207 ----------------------------------------------------------------------------------------------------
2020-06-23 12:11:33,268 epoch 2 - iter 0/4 - loss 0.66289943 - samples/sec: 545.88
2020-06-23 12:11:33,315 epoch 2 - iter 1/4 - loss 0.71299613 - samples/sec: 795.10
2020-06-23 12:11:33,371 epoch 2 - iter 2/4 - loss 0.71684919 - samples/sec: 684.60
2020-06-23 12:11:33,392 epoch 2 - iter 3/4 - loss 0.59646435 - samples/sec: 3026.13
2020-06-23 12:11:33,398 ----------------------------------------------------------------------------------------------------
2020-06-23 12:11:33,399 EPOCH 2 done: loss 0.5965 - lr 0.1000
2020-06-23 12:11:35,585 DEV : loss 0.9560748934745789 - score 0.7667
2020-06-23 12:11:35,735 BAD EPOCHS (no improvement): 1
2020-06-23 12:11:36,845 ----------------------------------------------------------------------------------------------------
2020-06-23 12:11:36,892 epoch 3 - iter 0/4 - loss 1.06638706 - samples/sec: 708.91
2020-06-23 12:11:36,936

2020-06-23 12:12:01,006 ----------------------------------------------------------------------------------------------------
2020-06-23 12:12:01,007 EPOCH 12 done: loss 0.3100 - lr 0.0500
2020-06-23 12:12:03,293 DEV : loss 0.7280484437942505 - score 0.728
2020-06-23 12:12:03,440 BAD EPOCHS (no improvement): 2
2020-06-23 12:12:03,442 ----------------------------------------------------------------------------------------------------
2020-06-23 12:12:03,480 epoch 13 - iter 0/4 - loss 0.25505537 - samples/sec: 868.18
2020-06-23 12:12:03,534 epoch 13 - iter 1/4 - loss 0.19914441 - samples/sec: 764.66
2020-06-23 12:12:03,584 epoch 13 - iter 2/4 - loss 0.24425759 - samples/sec: 774.43
2020-06-23 12:12:03,602 epoch 13 - iter 3/4 - loss 0.29874956 - samples/sec: 2942.53
2020-06-23 12:12:03,609 ----------------------------------------------------------------------------------------------------
2020-06-23 12:12:03,610 EPOCH 13 done: loss 0.2987 - lr 0.0500
2020-06-23 12:12:05,907 DEV : loss 0.81

In [34]:
# Flair
# https://github.com/flairNLP/flair/blob/master/resources/docs/embeddings/FLAIR_EMBEDDINGS.md
total_time = time.time()
word_embeddings = [ # FlairEmbeddings('multi-forward'), # this is 300 languge, gave very low score
                  # FlairEmbeddings('multi-backward'), 
                   FlairEmbeddings('news-forward'),  #  	English 	Trained with 1 billion word corpus                   
                   
                  ]
#modelname = 'Flair-news-fwd'
modelname = 'Flair'
train_and_predict(word_embeddings, modelname, modeldesc, savelist=savelist, epochs=EPOCHS)

print(time.time() - total_time)

# 167s in 100 train
# 620s for 18k

Flair
2020-05-15 16:16:53,990 Reading data from /home/max/git/newcombined/dataset_hatespeech/input
2020-05-15 16:16:53,991 Train: /home/max/git/newcombined/dataset_hatespeech/input/flair_train.csv
2020-05-15 16:16:53,992 Dev: /home/max/git/newcombined/dataset_hatespeech/input/flair_dev.csv
2020-05-15 16:16:53,993 Test: /home/max/git/newcombined/dataset_hatespeech/input/flair_test.csv


/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:452: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, tokenizer=tokenizer, max_tokens_per_doc=max_tokens_per_doc
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:457: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, tokenizer=tokenizer, max_tokens_per_doc=max_tokens_per_doc
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:464: DeprecationWarning:

2020-05-15 16:16:55,465 Computing label dictionary. Progress:


100%|██████████| 100/100 [00:00<00:00, 92040.90it/s]

2020-05-15 16:16:55,468 [b'1', b'2', b'0']
2020-05-15 16:16:55,469 Computing label dictionary. Progress:


...

2020-05-15 16:16:55,473 [b'1', b'2', b'0']
2020-05-15 16:16:55,477 ----------------------------------------------------------------------------------------------------
2020-05-15 16:16:55,478 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=300, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2048, out_features=256, bias=True)
    (rnn): LSTM(256, 512, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=3, bias=True)
  (loss_function): CrossEntropyLoss()
)"
2020-05-15 16:16:55,479 ----------------------------------------------------------------------------------------------------

2020-05-15 16:16:55,788 epoch 1 - iter 1/4 - loss 1.03974864 - samples/sec: 199.91
2020-05-15 16:16:55,934 epoch 1 - iter 2/4 - loss 1.02089870 - samples/sec: 234.98
2020-05-15 16:16:55,987 epoch 1 - iter 3/4 - loss 0.99495506 - samples/sec: 766.80
2020-05-15 16:16:55,997 ----------------------------------------------------------------------------------------------------
2020-05-15 16:16:55,998 EPOCH 1 done: loss 0.9950 - lr 0.1000
2020-05-15 16:17:07,322 DEV : loss 0.8543857336044312 - score 0.7667
2020-05-15 16:17:07,459 BAD EPOCHS (no improvement): 0


/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LanguageModel. It won't

2020-05-15 16:17:07,736 ----------------------------------------------------------------------------------------------------
2020-05-15 16:17:07,793 epoch 2 - iter 0/4 - loss 0.83632404 - samples/sec: 591.07
2020-05-15 16:17:07,840 epoch 2 - iter 1/4 - loss 0.85096899 - samples/sec: 889.78
2020-05-15 16:17:07,890 epoch 2 - iter 2/4 - loss 0.83957122 - samples/sec: 824.34
2020-05-15 16:17:07,915 epoch 2 - iter 3/4 - loss 0.76601796 - samples/sec: 3255.42
2020-05-15 16:17:07,930 ----------------------------------------------------------------------------------------------------
2020-05-15 16:17:07,931 EPOCH 2 done: loss 0.7660 - lr 0.1000
2020-05-15 16:17:09,805 DEV : loss 0.7338841557502747 - score 0.7667
2020-05-15 16:17:09,942 BAD EPOCHS (no improvement): 1
2020-05-15 16:17:10,213 ----------------------------------------------------------------------------------------------------
2020-05-15 16:17:10,261 epoch 3 - iter 0/4 - loss 0.81052619 - samples/sec: 709.31
2020-05-15 16:17:10,310

2020-05-15 16:17:33,044 ----------------------------------------------------------------------------------------------------
2020-05-15 16:17:33,045 EPOCH 12 done: loss 0.6617 - lr 0.0500
2020-05-15 16:17:34,992 DEV : loss 0.6712132692337036 - score 0.7667
2020-05-15 16:17:35,126 BAD EPOCHS (no improvement): 5
2020-05-15 16:17:35,379 ----------------------------------------------------------------------------------------------------
2020-05-15 16:17:35,419 epoch 13 - iter 0/4 - loss 0.65164173 - samples/sec: 849.37
2020-05-15 16:17:35,474 epoch 13 - iter 1/4 - loss 0.63884535 - samples/sec: 759.30
2020-05-15 16:17:35,526 epoch 13 - iter 2/4 - loss 0.66198879 - samples/sec: 766.89
2020-05-15 16:17:35,552 epoch 13 - iter 3/4 - loss 0.74194731 - samples/sec: 2931.03
2020-05-15 16:17:35,567 ----------------------------------------------------------------------------------------------------
2020-05-15 16:17:35,568 EPOCH 13 done: loss 0.7419 - lr 0.0500
2020-05-15 16:17:37,529 DEV : loss 0.6

In [35]:
# BERT OOMs on 32 batch, use 8
total_time = time.time()

# Bert Cased - separating lower and upper case, uncased - ignoring case.
# Use cased

word_embeddings = [ BertEmbeddings('bert-base-cased'),                ]
modelname = 'bert-base-cased'
train_and_predict(word_embeddings, modelname, modeldesc, savelist=savelist, epochs=EPOCHS, batch_size=8)

#word_embeddings = [ BertEmbeddings('bert-base-uncased'),                ]
#modelname = 'bert-base-uncased'
#train_and_predict(word_embeddings, modelname, modeldesc, savelist=savelist, epochs=EPOCHS, batch_size=8)

#BPE - takes memory - reduce batch size radically!
word_embeddings = [ BytePairEmbeddings('en'),   ]
modelname = 'BytePairEmbedding'
train_and_predict(word_embeddings, modelname, modeldesc, savelist=savelist, epochs=EPOCHS, batch_size=8)               

print(time.time() - total_time)

# 450 s 18k

bert-base-cased
2020-05-15 16:19:28,381 Reading data from /home/max/git/newcombined/dataset_hatespeech/input
2020-05-15 16:19:28,382 Train: /home/max/git/newcombined/dataset_hatespeech/input/flair_train.csv
2020-05-15 16:19:28,383 Dev: /home/max/git/newcombined/dataset_hatespeech/input/flair_dev.csv
2020-05-15 16:19:28,383 Test: /home/max/git/newcombined/dataset_hatespeech/input/flair_test.csv


/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:452: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, tokenizer=tokenizer, max_tokens_per_doc=max_tokens_per_doc
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:457: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, tokenizer=tokenizer, max_tokens_per_doc=max_tokens_per_doc
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:464: DeprecationWarning:

2020-05-15 16:19:29,825 Computing label dictionary. Progress:


100%|██████████| 100/100 [00:00<00:00, 214542.40it/s]

2020-05-15 16:19:29,828 [b'1', b'2', b'0']
2020-05-15 16:19:29,829 Computing label dictionary. Progress:


...

2020-05-15 16:25:12,230 ----------------------------------------------------------------------------------------------------
2020-05-15 16:25:12,253 epoch 2 - iter 0/13 - loss 0.79609919 - samples/sec: 389.72
2020-05-15 16:25:12,274 epoch 2 - iter 1/13 - loss 0.55464861 - samples/sec: 517.61
2020-05-15 16:25:12,302 epoch 2 - iter 2/13 - loss 0.55214993 - samples/sec: 376.95
2020-05-15 16:25:12,324 epoch 2 - iter 3/13 - loss 0.60946737 - samples/sec: 500.39


/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


2020-05-15 16:25:12,352 epoch 2 - iter 4/13 - loss 0.64863864 - samples/sec: 510.80
2020-05-15 16:25:12,374 epoch 2 - iter 5/13 - loss 0.65376938 - samples/sec: 508.08
2020-05-15 16:25:12,401 epoch 2 - iter 6/13 - loss 0.62946188 - samples/sec: 441.76
2020-05-15 16:25:12,422 epoch 2 - iter 7/13 - loss 0.67950963 - samples/sec: 516.87
2020-05-15 16:25:12,449 epoch 2 - iter 8/13 - loss 0.68462497 - samples/sec: 386.93
2020-05-15 16:25:12,474 epoch 2 - iter 9/13 - loss 0.66254631 - samples/sec: 429.56
2020-05-15 16:25:12,498 epoch 2 - iter 10/13 - loss 0.64845571 - samples/sec: 526.15
2020-05-15 16:25:12,527 epoch 2 - iter 11/13 - loss 0.67873405 - samples/sec: 482.43
2020-05-15 16:25:12,548 epoch 2 - iter 12/13 - loss 0.65932834 - samples/sec: 797.81
2020-05-15 16:25:12,554 ----------------------------------------------------------------------------------------------------
2020-05-15 16:25:12,555 EPOCH 2 done: loss 0.6593 - lr 0.1000
2020-05-15 16:25:14,878 DEV : loss 0.6721192002296448 

2020-05-15 16:25:28,307 epoch 7 - iter 11/13 - loss 0.58229414 - samples/sec: 608.88
2020-05-15 16:25:28,324 epoch 7 - iter 12/13 - loss 0.57068111 - samples/sec: 758.17
2020-05-15 16:25:28,334 ----------------------------------------------------------------------------------------------------
2020-05-15 16:25:28,335 EPOCH 7 done: loss 0.5707 - lr 0.1000
2020-05-15 16:25:31,000 DEV : loss 0.6939222812652588 - score 0.7663
2020-05-15 16:25:31,149 BAD EPOCHS (no improvement): 3
2020-05-15 16:25:31,150 ----------------------------------------------------------------------------------------------------
2020-05-15 16:25:31,167 epoch 8 - iter 0/13 - loss 0.29959819 - samples/sec: 524.85
2020-05-15 16:25:31,194 epoch 8 - iter 1/13 - loss 0.52598937 - samples/sec: 502.93
2020-05-15 16:25:31,220 epoch 8 - iter 2/13 - loss 0.44922308 - samples/sec: 540.19
2020-05-15 16:25:31,244 epoch 8 - iter 3/13 - loss 0.57538338 - samples/sec: 456.77
2020-05-15 16:25:31,271 epoch 8 - iter 4/13 - loss 0.50351

2020-05-15 16:25:46,051 ----------------------------------------------------------------------------------------------------
2020-05-15 16:25:46,067 epoch 13 - iter 0/13 - loss 0.71879673 - samples/sec: 563.31
2020-05-15 16:25:46,088 epoch 13 - iter 1/13 - loss 0.49519822 - samples/sec: 558.13
2020-05-15 16:25:46,107 epoch 13 - iter 2/13 - loss 0.44308680 - samples/sec: 692.49
2020-05-15 16:25:46,127 epoch 13 - iter 3/13 - loss 0.49287412 - samples/sec: 601.19
2020-05-15 16:25:46,160 epoch 13 - iter 4/13 - loss 0.49923037 - samples/sec: 434.40
2020-05-15 16:25:46,183 epoch 13 - iter 5/13 - loss 0.45436286 - samples/sec: 489.58
2020-05-15 16:25:46,212 epoch 13 - iter 6/13 - loss 0.44483879 - samples/sec: 372.55
2020-05-15 16:25:46,243 epoch 13 - iter 7/13 - loss 0.46565047 - samples/sec: 409.26
2020-05-15 16:25:46,268 epoch 13 - iter 8/13 - loss 0.47630741 - samples/sec: 586.07
2020-05-15 16:25:46,297 epoch 13 - iter 9/13 - loss 0.46855751 - samples/sec: 358.03
2020-05-15 16:25:46,328 e

In [36]:
# GPT-1

total_time = time.time()
#Do we need batchsize 8 here?

word_embeddings = [ OpenAIGPTEmbeddings(),                ]
modelname = 'gpt-1'
train_and_predict(word_embeddings, modelname, modeldesc, savelist=savelist, epochs=EPOCHS, batch_size=8)

print(time.time() - total_time)

# 302 sec 18k

gpt-1
2020-05-15 16:26:16,286 Reading data from /home/max/git/newcombined/dataset_hatespeech/input
2020-05-15 16:26:16,286 Train: /home/max/git/newcombined/dataset_hatespeech/input/flair_train.csv
2020-05-15 16:26:16,287 Dev: /home/max/git/newcombined/dataset_hatespeech/input/flair_dev.csv
2020-05-15 16:26:16,287 Test: /home/max/git/newcombined/dataset_hatespeech/input/flair_test.csv


/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:452: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, tokenizer=tokenizer, max_tokens_per_doc=max_tokens_per_doc
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:457: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, tokenizer=tokenizer, max_tokens_per_doc=max_tokens_per_doc
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/flair/data_fetcher.py:464: DeprecationWarning:

2020-05-15 16:26:17,748 Computing label dictionary. Progress:


100%|██████████| 100/100 [00:00<00:00, 205200.78it/s]

2020-05-15 16:26:17,751 [b'1', b'2', b'0']
2020-05-15 16:26:17,752 Computing label dictionary. Progress:


...

2020-05-15 16:26:18,228 epoch 1 - iter 1/13 - loss 1.02366441 - samples/sec: 46.87
2020-05-15 16:26:18,507 epoch 1 - iter 2/13 - loss 0.89995372 - samples/sec: 44.56
2020-05-15 16:26:18,777 epoch 1 - iter 3/13 - loss 0.89521576 - samples/sec: 49.18
2020-05-15 16:26:19,048 epoch 1 - iter 4/13 - loss 0.90249537 - samples/sec: 48.36
2020-05-15 16:26:19,305 epoch 1 - iter 5/13 - loss 0.86521447 - samples/sec: 50.51
2020-05-15 16:26:19,589 epoch 1 - iter 6/13 - loss 0.80853843 - samples/sec: 43.72
2020-05-15 16:26:19,881 epoch 1 - iter 7/13 - loss 0.76403137 - samples/sec: 44.48
2020-05-15 16:26:20,148 epoch 1 - iter 8/13 - loss 0.76629166 - samples/sec: 48.27
2020-05-15 16:26:20,410 epoch 1 - iter 9/13 - loss 0.77882037 - samples/sec: 49.42
2020-05-15 16:26:20,675 epoch 1 - iter 10/13 - loss 0.79162548 - samples/sec: 49.06
2020-05-15 16:26:20,932 epoch 1 - iter 11/13 - loss 0.75281586 - samples/sec: 50.92
2020-05-15 16:26:21,111 epoch 1 - iter 12/13 - loss 0.76204223 - samples/sec: 106.16


/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type OpenAIGPTEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/max/anaconda3/envs/tf-hub/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type OpenAIGPTModel. It 

2020-05-15 16:27:23,673 ----------------------------------------------------------------------------------------------------
2020-05-15 16:27:23,696 epoch 2 - iter 0/13 - loss 0.63897491 - samples/sec: 430.39
2020-05-15 16:27:23,818 epoch 2 - iter 1/13 - loss 0.45528595 - samples/sec: 486.47
2020-05-15 16:27:23,939 epoch 2 - iter 2/13 - loss 0.49192606 - samples/sec: 506.12
2020-05-15 16:27:24,061 epoch 2 - iter 3/13 - loss 0.53844932 - samples/sec: 460.98
2020-05-15 16:27:24,183 epoch 2 - iter 4/13 - loss 0.56347318 - samples/sec: 528.67
2020-05-15 16:27:24,303 epoch 2 - iter 5/13 - loss 0.58326705 - samples/sec: 600.20
2020-05-15 16:27:24,431 epoch 2 - iter 6/13 - loss 0.57723636 - samples/sec: 438.36
2020-05-15 16:27:24,552 epoch 2 - iter 7/13 - loss 0.60624900 - samples/sec: 466.90
2020-05-15 16:27:24,681 epoch 2 - iter 8/13 - loss 0.61534938 - samples/sec: 406.37
2020-05-15 16:27:24,807 epoch 2 - iter 9/13 - loss 0.59594664 - samples/sec: 554.91
2020-05-15 16:27:24,928 epoch 2 - i

2020-05-15 16:27:50,242 epoch 7 - iter 5/13 - loss 0.21901545 - samples/sec: 552.09
2020-05-15 16:27:50,364 epoch 7 - iter 6/13 - loss 0.23229106 - samples/sec: 477.70
2020-05-15 16:27:50,484 epoch 7 - iter 7/13 - loss 0.26107138 - samples/sec: 574.04
2020-05-15 16:27:50,604 epoch 7 - iter 8/13 - loss 0.26079476 - samples/sec: 486.37
2020-05-15 16:27:50,728 epoch 7 - iter 9/13 - loss 0.26099357 - samples/sec: 429.84
2020-05-15 16:27:50,849 epoch 7 - iter 10/13 - loss 0.24235101 - samples/sec: 491.15
2020-05-15 16:27:50,967 epoch 7 - iter 11/13 - loss 0.27669199 - samples/sec: 563.88
2020-05-15 16:27:51,107 epoch 7 - iter 12/13 - loss 0.27300339 - samples/sec: 554.26
2020-05-15 16:27:51,266 ----------------------------------------------------------------------------------------------------
2020-05-15 16:27:51,267 EPOCH 7 done: loss 0.2730 - lr 0.1000
2020-05-15 16:27:54,252 DEV : loss 0.688732922077179 - score 0.7683
2020-05-15 16:27:54,388 BAD EPOCHS (no improvement): 2
2020-05-15 16:2

2020-05-15 16:28:14,586 epoch 12 - iter 11/13 - loss 0.12967260 - samples/sec: 433.34
2020-05-15 16:28:14,703 epoch 12 - iter 12/13 - loss 0.14627472 - samples/sec: 436.42
2020-05-15 16:28:14,803 ----------------------------------------------------------------------------------------------------
2020-05-15 16:28:14,803 EPOCH 12 done: loss 0.1463 - lr 0.1000
2020-05-15 16:28:17,459 DEV : loss 1.060694932937622 - score 0.7683
2020-05-15 16:28:17,601 BAD EPOCHS (no improvement): 1
2020-05-15 16:28:17,602 ----------------------------------------------------------------------------------------------------
2020-05-15 16:28:17,622 epoch 13 - iter 0/13 - loss 0.19827484 - samples/sec: 463.82
2020-05-15 16:28:17,741 epoch 13 - iter 1/13 - loss 0.10357209 - samples/sec: 474.06
2020-05-15 16:28:17,863 epoch 13 - iter 2/13 - loss 0.08482356 - samples/sec: 484.10
2020-05-15 16:28:18,018 epoch 13 - iter 3/13 - loss 0.06928484 - samples/sec: 460.17
2020-05-15 16:28:18,137 epoch 13 - iter 4/13 - loss 

### SAVE

In [37]:
#name='all_flair_512LSTM_15ep_8model'
name='FINAL_flair_all_trainsz_'+str(trainsize)
print(len(savelist))
saveResults(savelist, name=name)

9


In [38]:
len(savelist)

9

In [39]:
trainsize

100

### DONE

In [ ]:
# Code for combining results from 2 training times into 1 list etc.

In [ ]:
# add the 2 last
name='all_flair_512LSTM_15ep_model'
name = 'FINAL_flair_all_trainsz_'+str(trainsize)
te = loadResults(name)

In [145]:
#newlist = te+savelist
newlist = te
len(newlist)

9

In [146]:
# cut doubles away
# newlist = newlist[5:]  

In [147]:
# save combined
name='FINAL_flair_all_trainsz_'+str(trainsize)
print(len(newlist))
saveResults(newlist, name=name)

9


In [ ]:
# move list item 5 to end
# te[5]
# te.append(te.pop(5))


In [41]:
len(savelist)

10

In [69]:
te = savelist[0:5] + [savelist[9]] + savelist[6:9]
len(te)

9

In [131]:
for i in savelist:
    print(i['model'])

Flair
bert-base-uncased


In [139]:
for i in te:
    print(i['model'])

glove
fasttext web-crawl
fasttext news/wiki
en-twitter
elmo
Flair
bert-base-uncased
BytePairEmbedding
gpt-1


In [75]:
savelist[9]

{'model': 'Flair', 'labels': 0       1
 1       1
 2       1
 3       1
 4       1
        ..
 2995    1
 2996    1
 2997    1
 2998    1
 2999    1
 Name: label, Length: 3000, dtype: int64, 'confidence': 0       0.876461
 1       0.711614
 2       0.925566
 3       0.895800
 4       0.885994
           ...   
 2995    0.751575
 2996    0.940553
 2997    0.902379
 2998    0.835532
 2999    0.964116
 Name: confidence, Length: 3000, dtype: float64, 'traintime': 125.29585218429565, 'predtime3k': 1589372012.3927588, 'modeldesc': '512LSTM_15epoch_non-bi'}

In [130]:
savelist[0]['model']='Flair'

In [132]:
trainsize

1000

In [140]:
te[5]

{'model': 'Flair', 'labels': 0       1
 1       1
 2       1
 3       1
 4       1
        ..
 2995    1
 2996    1
 2997    1
 2998    1
 2999    1
 Name: label, Length: 3000, dtype: int64, 'confidence': 0       0.805744
 1       0.546344
 2       0.802081
 3       0.714745
 4       0.753530
           ...   
 2995    0.759887
 2996    0.893057
 2997    0.859725
 2998    0.813483
 2999    0.864573
 Name: confidence, Length: 3000, dtype: float64, 'traintime': 81.63569831848145, 'predtime3k': 1589378399.4414687, 'modeldesc': '512LSTM_15epoch_non-bi'}

In [138]:
te[5] = savelist[0]

In [142]:
te[6] = savelist[1]

In [149]:
for i in newlist:
    print(i['model'])

glove
fasttext web-crawl
fasttext news/wiki
en-twitter
elmo
Flair
bert-base-uncased
BytePairEmbedding
gpt-1


In [150]:
savelist[1]

{'model': 'bert-base-uncased', 'labels': 0       1
 1       1
 2       1
 3       1
 4       1
        ..
 2995    1
 2996    2
 2997    2
 2998    2
 2999    1
 Name: label, Length: 3000, dtype: int64, 'confidence': 0       0.996391
 1       0.995925
 2       0.740776
 3       0.996946
 4       0.999737
           ...   
 2995    0.997924
 2996    0.775749
 2997    0.926272
 2998    0.993099
 2999    0.947747
 Name: confidence, Length: 3000, dtype: float64, 'traintime': 273.37105560302734, 'predtime3k': 1589378548.903775, 'modeldesc': '512LSTM_15epoch_non-bi'}